In [5]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.5
batch_size = 100
n_epochs = 51
delta = 1.0
hidden_size = 10
# regulation_rate = 1e-4

def fullLayer(input_data, output_size, act, name, std = 0.1):
    ##defining the full linear Layer here
    w = tf.Variable(tf.random_normal([input_data.get_shape().as_list()[1], output_size], stddev = std, name = name + "_weigth"))
#     w = tf.Variable(tf.zeros([input_data.shape[1], output_size]))
    b = tf.Variable(tf.zeros([output_size]), name = name + "_bias")
    return act(tf.matmul(input_data, w) + b), w, b

#read data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#define placeholder
# all image are 28 * 28 so x has 784 dimension
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
Y = tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')
stddev = tf.Variable(0.1, name = "stddev")

## the layers
h, w1, b1 = fullLayer(X, hidden_size, tf.nn.sigmoid, "layer_1", stddev)
logits, w2, b2 = fullLayer(h, 10, tf.nn.sigmoid, "layer_2", stddev)

## defining loss function
## use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
## computes the mean over all the examples in the batch
loss = tf.reduce_mean(entropy) 
# + regulation_rate*tf.nn.l2_loss(w)  

##defining optimizer
## using gradient descent with learning rate of 0.5 to minimize loss
gradient = tf.train.GradientDescentOptimizer(learning_rate)
optimizer = gradient.minimize(loss)

##the prediction we made
preds = tf.nn.softmax(logits)
##check how many of them are correct arg maxx is used because Y is one hat
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

#just some config for not getting whole server
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options)

with tf.Session(config = config) as sess:
        
    # to visualize using TensorBoard
    ## making the graph of the network as b) demands
#     writer = tf.summary.FileWriter('./graphs/mnist/a_2')
    
    for std in [0.2, 2.0]:
        ##training the model 
        
#         tf.summary.histogram("hidden_weigths",tf.reduce_mean(w1))
#         merge=tf.summary.merge_all()
        ##starting time
        start_time = time.time()
        ##initialize the variables
        sess.run(tf.global_variables_initializer())
        ##asign stddev
        sess.run(stddev.assign(std))

        
        ## number of training batches
        n_batches = int(mnist.train.num_examples / batch_size)
        
        for i in range(n_epochs):  # train the model n_epochs times
            total_loss = 0
            total_acc = 0
            for _ in range(n_batches):
                ##training batches
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                _, loss_batch, acc_batch = sess.run([optimizer, loss, accuracy], feed_dict={X: X_batch, Y: Y_batch})
                total_loss += loss_batch
                total_acc += acc_batch
                
#                 writer.add_summary(w,std)
            if i % 5 == 0:
                print('with stddev {}, epoch {}, Average loss : {}, Accuracy : {:.6f}'.format(std, i, total_loss / n_batches, total_acc / mnist.train.num_examples))

        print('Total time: {0} seconds'.format(time.time() - start_time))
        print('Optimization Finished!')
        
        # test the model
        ##number of test batches
        n_batches = int(mnist.test.num_examples / batch_size)
        total_correct_preds = 0

        for i in range(n_batches):
            ##test batches
            X_batch, Y_batch = mnist.test.next_batch(batch_size)
            accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y: Y_batch})
            total_correct_preds += accuracy_batch[0]

        print('Accuracy stddev {}, {}'.format(std, total_correct_preds / mnist.test.num_examples))
        
#     writer.close()
sess.close()

'''
همانطور که از نمودار مشخص است انگار با انحراف معیار 2 وزن ها در حال کاهش یافتن هستند
اما وقتی انحراف معیار 0.2 است این اتفاق نمی افتد
می دانیم که خروجی سیگموید نزدیکی 0 شبیه خظی است اما در فاصله های دور تر به یک میل میکند
پس اگر وزن ها زیادی بزرگ یا کوچک باشند جواب هاب کم تری را شبکه میتواند بپوشاند
پس در فرآیند یادگیری در حالتی که وزن ها زیاداند کاهش میابند
'''

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
with stddev 0.2, epoch 0, Average loss : 2.075164391560988, Accuracy : 0.542473
with stddev 0.2, epoch 5, Average loss : 1.6368500380082565, Accuracy : 0.822909
with stddev 0.2, epoch 10, Average loss : 1.5992591586979952, Accuracy : 0.838509
with stddev 0.2, epoch 15, Average loss : 1.5840850327231668, Accuracy : 0.846327
with stddev 0.2, epoch 20, Average loss : 1.575557921366258, Accuracy : 0.850600
with stddev 0.2, epoch 25, Average loss : 1.5700109481811524, Accuracy : 0.852455
with stddev 0.2, epoch 30, Average loss : 1.5659259674765846, Accuracy : 0.853782
with stddev 0.2, epoch 35, Average loss : 1.562600990642201, Accuracy : 0.851309
with stddev 0.2, epoch 40, Average loss : 1.5586661176248031, Accuracy : 0.839236
with stddev 0.2, epoch 45, Average loss : 1.5547406846826726, Accuracy : 

'\nهمانطور که از نمودار مشخص است انگار با انحراف معیار 2\nوزن ها کاهش می\n'